In [1]:
from sqlalchemy import create_engine
from config import password
import pymysql
pymysql.install_as_MySQLdb()
import pandas as pd

# Extract

In [2]:
#Load and display hpsa csv
hpsa_file = "./Resources/Primary Dataset - HPSA.csv"
hpsa_df = pd.read_csv(hpsa_file)
hpsa_df.head()

,DP_PRIMARY_ID,HPSA Name,HPSA ID,Designation Type,HPSA Discipline Class,HPSA Score,Primary State Abbreviation,HPSA Status,HPSA Designation Date,County Name,ZIP Code,Longitude,Latitude,Unnamed: 13
0,112999128P,West Florida Medical Associates,112999128P,Rural Health Clinic,Primary Care,1,FL,Designated,09/30/2006,Levy County,34449-9463,-82.671644,29.031793,NaN
1,712999121L,Suncoast Community Health Centers,712999121L,Federally Qualified Health Center,Mental Health,14,FL,Designated,11/14/2003,Hillsborough County,33570-5471,-82.394713,27.705737,NaN
2,112999126V,Northeast Florida Health Services,112999126V,Federally Qualified Health Center,Primary Care,14,FL,Designated,09/30/2003,Volusia County,32180-2288,-81.467121,29.240109,NaN
3,112999121P,"Camillus Health Concern, Inc.",112999121P,Federally Qualified Health Center,Primary Care,12,FL,Designated,11/14/2003,Miami-Dade County,33128-1616,-80.200343,25.778733,NaN
4,112999121L,Center for Family and Child Enrichment,112999121L,Federally Qualified Health Center,Primary Care,17,FL,Designated,11/01/2013,Miami-Dade County,33056-4838,-80.231085,25.927185,NaN


In [3]:
#Load and display uninsured csv
uninsured_file = "./Resources/ACS_17_5YR_S2702_with_ann.csv"
uninsured_df = pd.read_csv(uninsured_file)
uninsured_df.head()

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC01_EST_VC04,HC01_MOE_VC04,HC02_EST_VC04,...,HC02_EST_VC138,HC02_MOE_VC138,HC01_EST_VC139,HC01_MOE_VC139,HC02_EST_VC139,HC02_MOE_VC139,HC01_EST_VC140,HC01_MOE_VC140,HC02_EST_VC140,HC02_MOE_VC140
0,Id,Id2,Geography,Total; Estimate; Total civilian noninstitution...,Total; Margin of Error; Total civilian noninst...,Total Uninsured; Estimate; Total civilian noni...,Total Uninsured; Margin of Error; Total civili...,Total; Estimate; AGE - Under 19 years,Total; Margin of Error; AGE - Under 19 years,Total Uninsured; Estimate; AGE - Under 19 years,...,Total Uninsured; Estimate; 150 to 199 percent ...,Total Uninsured; Margin of Error; 150 to 199 p...,Total; Estimate; 200 to 299 percent of the pov...,Total; Margin of Error; 200 to 299 percent of ...,Total Uninsured; Estimate; 200 to 299 percent ...,Total Uninsured; Margin of Error; 200 to 299 p...,Total; Estimate; At or above 300 percent of th...,Total; Margin of Error; At or above 300 percen...,Total Uninsured; Estimate; At or above 300 per...,Total Uninsured; Margin of Error; At or above ...
1,0400000US12,12,Florida,19967931,1615,2982945,31700,21.8,0.1,12.4,...,14.3,0.3,18.1,0.1,20.3,0.3,45.4,0.2,22.9,0.3
2,0500000US12001,12001,"Alachua County, Florida",257247,255,27457,1596,21.1,0.2,9.4,...,10.9,1.9,14.7,0.9,15.3,2.0,44.1,1.2,20.3,2.5
3,0500000US12005,12005,"Bay County, Florida",175626,485,26312,1581,23.3,0.1,14.1,...,14.1,2.4,18.8,1.3,21.9,2.6,44.8,1.6,23.5,3.1
4,0500000US12009,12009,"Brevard County, Florida",563561,451,67853,2691,19.9,0.1,10.9,...,15.5,1.8,18.4,0.8,18.0,1.7,48.6,0.8,25.9,2.0


# Transform

In [4]:
#Get relevant info and rename columns
reduced_hpsa_df = hpsa_df[["HPSA Name", "Designation Type", "HPSA Discipline Class","HPSA Designation Date","County Name","ZIP Code","Longitude","Latitude"]]
rename_hpsa_df = reduced_hpsa_df.rename(columns= {"HPSA Name":"hpsa_name","Designation Type":"designation_type","HPSA Discipline Class":"hpsa_discipline_class","HPSA Designation Date":"hpsa_designation_date","County Name":"county_name","ZIP Code":"zip_code","Longitude":"longitude", "Latitude":"latitude"})
rename_hpsa_df.head()

,hpsa_name,designation_type,hpsa_discipline_class,hpsa_designation_date,county_name,zip_code,longitude,latitude
0,West Florida Medical Associates,Rural Health Clinic,Primary Care,09/30/2006,Levy County,34449-9463,-82.671644,29.031793
1,Suncoast Community Health Centers,Federally Qualified Health Center,Mental Health,11/14/2003,Hillsborough County,33570-5471,-82.394713,27.705737
2,Northeast Florida Health Services,Federally Qualified Health Center,Primary Care,09/30/2003,Volusia County,32180-2288,-81.467121,29.240109
3,"Camillus Health Concern, Inc.",Federally Qualified Health Center,Primary Care,11/14/2003,Miami-Dade County,33128-1616,-80.200343,25.778733
4,Center for Family and Child Enrichment,Federally Qualified Health Center,Primary Care,11/01/2013,Miami-Dade County,33056-4838,-80.231085,25.927185


In [5]:
#removing the fields that lack physical location
clean_hpsa_df= rename_hpsa_df.dropna(how='any')
clean_hpsa_df.head()

,hpsa_name,designation_type,hpsa_discipline_class,hpsa_designation_date,county_name,zip_code,longitude,latitude
0,West Florida Medical Associates,Rural Health Clinic,Primary Care,09/30/2006,Levy County,34449-9463,-82.671644,29.031793
1,Suncoast Community Health Centers,Federally Qualified Health Center,Mental Health,11/14/2003,Hillsborough County,33570-5471,-82.394713,27.705737
2,Northeast Florida Health Services,Federally Qualified Health Center,Primary Care,09/30/2003,Volusia County,32180-2288,-81.467121,29.240109
3,"Camillus Health Concern, Inc.",Federally Qualified Health Center,Primary Care,11/14/2003,Miami-Dade County,33128-1616,-80.200343,25.778733
4,Center for Family and Child Enrichment,Federally Qualified Health Center,Primary Care,11/01/2013,Miami-Dade County,33056-4838,-80.231085,25.927185


In [6]:
#Remove state from county and make new column with just county(last column)
uninsured_df['county_name'] = uninsured_df['GEO.display-label'].str.split(',').str[0]
uninsured_df.head()

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC01_EST_VC04,HC01_MOE_VC04,HC02_EST_VC04,...,HC02_MOE_VC138,HC01_EST_VC139,HC01_MOE_VC139,HC02_EST_VC139,HC02_MOE_VC139,HC01_EST_VC140,HC01_MOE_VC140,HC02_EST_VC140,HC02_MOE_VC140,county_name
0,Id,Id2,Geography,Total; Estimate; Total civilian noninstitution...,Total; Margin of Error; Total civilian noninst...,Total Uninsured; Estimate; Total civilian noni...,Total Uninsured; Margin of Error; Total civili...,Total; Estimate; AGE - Under 19 years,Total; Margin of Error; AGE - Under 19 years,Total Uninsured; Estimate; AGE - Under 19 years,...,Total Uninsured; Margin of Error; 150 to 199 p...,Total; Estimate; 200 to 299 percent of the pov...,Total; Margin of Error; 200 to 299 percent of ...,Total Uninsured; Estimate; 200 to 299 percent ...,Total Uninsured; Margin of Error; 200 to 299 p...,Total; Estimate; At or above 300 percent of th...,Total; Margin of Error; At or above 300 percen...,Total Uninsured; Estimate; At or above 300 per...,Total Uninsured; Margin of Error; At or above ...,Geography
1,0400000US12,12,Florida,19967931,1615,2982945,31700,21.8,0.1,12.4,...,0.3,18.1,0.1,20.3,0.3,45.4,0.2,22.9,0.3,Florida
2,0500000US12001,12001,"Alachua County, Florida",257247,255,27457,1596,21.1,0.2,9.4,...,1.9,14.7,0.9,15.3,2.0,44.1,1.2,20.3,2.5,Alachua County
3,0500000US12005,12005,"Bay County, Florida",175626,485,26312,1581,23.3,0.1,14.1,...,2.4,18.8,1.3,21.9,2.6,44.8,1.6,23.5,3.1,Bay County
4,0500000US12009,12009,"Brevard County, Florida",563561,451,67853,2691,19.9,0.1,10.9,...,1.8,18.4,0.8,18.0,1.7,48.6,0.8,25.9,2.0,Brevard County


In [7]:
#Reduce df to relevant info
reduced_df = uninsured_df.loc[:,["GEO.id2","county_name","HC01_EST_VC01","HC02_EST_VC01"]]

#Save totals for later use
totals = reduced_df.iloc[1]
totals
#Remove row with totals
drop_df = reduced_df.drop(1)
drop_df

#Rename the columns with info in first row
rename_list = reduced_df.iloc[0]
rename_list = list(rename_list)

old_columns = list(drop_df.columns)
rename_dict = {}
for i in range(len(old_columns)):
    rename_dict[old_columns[i]] = rename_list[i]

rename_df = drop_df.rename(columns=rename_dict)

#Rename again to fit into db
clean_uninsured_df = rename_df.drop(0).rename(columns={"Id2":"id2", "Geography":"county_name", "Total; Estimate; Total civilian noninstitutionalized population":"total_population","Total Uninsured; Estimate; Total civilian noninstitutionalized population":"total_uninsured_population"})
clean_uninsured_df.head()

,id2,county_name,total_population,total_uninsured_population
2,12001,Alachua County,257247,27457
3,12005,Bay County,175626,26312
4,12009,Brevard County,563561,67853
5,12011,Broward County,1878890,306930
6,12015,Charlotte County,169791,21578


# Load

In [8]:
#Connect to local database
rds_connection_string = f"root:{password}@localhost/medical_desert_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [9]:
#Check tables
engine.table_names()

['hpsa', 'uninsured', 'uninsured_rate']

In [10]:
#Load hpsa df into hpsa table in mysql
clean_hpsa_df.to_sql(name='hpsa', con=engine, if_exists='append', index=False)

In [11]:
#Preview hpsa table to verify load worked
pd.read_sql_query('select * from hpsa', con=engine).head()

,id,hpsa_name,designation_type,hpsa_discipline_class,hpsa_designation_date,county_name,zip_code,longitude,latitude
0,1,West Florida Medical Associates,Rural Health Clinic,Primary Care,09/30/2006,Levy County,34449-9463,-82.671644,29.031793
1,2,Suncoast Community Health Centers,Federally Qualified Health Center,Mental Health,11/14/2003,Hillsborough County,33570-5471,-82.394713,27.705737
2,3,Northeast Florida Health Services,Federally Qualified Health Center,Primary Care,09/30/2003,Volusia County,32180-2288,-81.467121,29.240109
3,4,"Camillus Health Concern, Inc.",Federally Qualified Health Center,Primary Care,11/14/2003,Miami-Dade County,33128-1616,-80.200343,25.778733
4,5,Center for Family and Child Enrichment,Federally Qualified Health Center,Primary Care,11/01/2013,Miami-Dade County,33056-4838,-80.231085,25.927185


In [12]:
#Load uninsured df into uninsured table in mysql
clean_uninsured_df.to_sql(name='uninsured', con=engine, if_exists='append', index=False)

In [13]:
#Preview uninsured table to verify load worked
pd.read_sql_query('select * from uninsured', con=engine).head()

,id,id2,county_name,total_population,total_uninsured_population
0,1,12001,Alachua County,257247,27457
1,2,12005,Bay County,175626,26312
2,3,12009,Brevard County,563561,67853
3,4,12011,Broward County,1878890,306930
4,5,12015,Charlotte County,169791,21578


In [14]:
#Join hpsa and uninsured by county
merger_table = pd.read_sql_query('SELECT h.county_name,h.longitude, h.latitude, u.total_population, u.total_uninsured_population FROM hpsa AS h JOIN uninsured AS u ON h.county_name = u.county_name', con=engine)

In [15]:
merger_table.head()

,county_name,longitude,latitude,total_population,total_uninsured_population
0,Hillsborough County,-82.394713,27.705737,1339107,186640
1,Volusia County,-81.467121,29.240109,513218,72783
2,Miami-Dade County,-80.200343,25.778733,2676045,552802
3,Miami-Dade County,-80.231085,25.927185,2676045,552802
4,Duval County,-81.687190,30.386283,892880,114630


In [16]:
#Calculate uninsured rate by county
merger_table['uninsured_rate'] = (merger_table['total_uninsured_population']/merger_table['total_population'])*100
merger_table.head()

,county_name,longitude,latitude,total_population,total_uninsured_population,uninsured_rate
0,Hillsborough County,-82.394713,27.705737,1339107,186640,13.937647
1,Volusia County,-81.467121,29.240109,513218,72783,14.181693
2,Miami-Dade County,-80.200343,25.778733,2676045,552802,20.657425
3,Miami-Dade County,-80.231085,25.927185,2676045,552802,20.657425
4,Duval County,-81.687190,30.386283,892880,114630,12.838231


In [17]:
#Remove county AND extra space before it
merger_table['NAME'] = merger_table['county_name'].str.split(' County').str[0]

#get relevant columns
final_uninsured_rate = merger_table.loc[:,['NAME',"uninsured_rate"]]
final_uninsured_rate.head()

,NAME,uninsured_rate
0,Hillsborough,13.937647
1,Volusia,14.181693
2,Miami-Dade,20.657425
3,Miami-Dade,20.657425
4,Duval,12.838231


In [18]:
#Load df into table
final_uninsured_rate.to_sql(name='uninsured_rate', con=engine, if_exists='append', index=False)

In [19]:
#Preview uninsure_rates table, Need this for choropleth map
pd.read_sql_query('select * from uninsured_rate', con=engine).head()

,id,NAME,uninsured_rate
0,1,Hillsborough,13.937647
1,2,Volusia,14.181693
2,3,Miami-Dade,20.657425
3,4,Miami-Dade,20.657425
4,5,Duval,12.838231
